In [20]:
##Logan Lauton
##Menu UI
##COSC 3810 - 101

##importing required packages for the menu's GUI    
from datetime import date
import datetime
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import getpass
from IPython.display import clear_output
import logging
import os
import oracledb
import pandas as pd
import requests
import smtplib
import sys
import time

##beginning with simple output showing a welcome statement for the menu 
print("Welcome to the Mental Health Database")
attempts = 0
##Startup Menu
while True:
    
    ##outputting one character at a time for look
    string = ("Input your Username:")
    for char in string:
        print(char, end='')
        time.sleep(.05)
    
    ##gathering username
    user = getpass.getpass()

    ##outputting one character at a time for look
    string = ("Input your Password:",)
    for char in string:
        print(char, end='')
        time.sleep(.05)
    
    ##gathering password
    password = getpass.getpass()
    ##number of login attempts allowed
    if attempts < 3:
        try:
            os.environ.get('TNS_ADMIN')
            connection = oracledb.connect(
                user=user,
                password=password,
                dsn = "ms73dyswf309gljx_high",
                config_dir="./config",
                wallet_location="./config",
                wallet_password="6YW?}$.5H(N*+vj6")
            print("Welcome " + user.title())
            break
        except Exception as e:
            print(e)
            time.sleep(3)
            clear_output()
            print("Try Re-entering")
            attempts = attempts + 1
            continue
    else:
        time.sleep(2)
        clear_output()
        print("You have reached the maximum number of attempts allowed.")
        email = input("Please enter your email address so that you can reset your password: ")
        subject = "Mentcare Patient Management System: Password Reset"
        body = """We have received a request to reset your password for your account with Mental Health Care Center. For security reasons, we require you to confirm this request by clicking the link below:

[Insert Password Reset Link Here]

If you did not initiate this password reset request, please contact our support team immediately at mentalcare.cosc.3810@gmail.com to report the issue.

Thank you for your attention to this matter.

Best regards,
Support
Mental Health Care Center"""


        msg = MIMEMultipart()
        msg['From'] = 'mentalcare.cosc.3810@gmail.com'
        msg['To'] = email
        msg['Subject'] = subject

        ##attach the email body as a plain text message
        msg.attach(MIMEText(body, 'plain'))

        smtp_server = smtplib.SMTP('smtp.gmail.com', 587) # Using the SMTP server of the email provider
        smtp_server.starttls()
        smtp_server.login('mentalcare.cosc.3810@gmail.com', 'password')  # Randomly created email that is going to be used
        ##NOTE: This email address will be deleted after the completion of the project

        smtp_server.send_message(msg)
        smtp_server.quit()

        print("Check you email")
        break


##check if to many login attempts occured, if not menu begins
if attempts < 3:

##ADMIN CASE ✅========================================================================================================
    if user.lower() == "admin":
        
        os.environ.get('TNS_ADMIN')
        connection = oracledb.connect(
            user='admin',
            password='8&n:f6=$uMrFp+j>',
            dsn = "ms73dyswf309gljx_high",
            config_dir="./config",
            wallet_location="./config",
            wallet_password="6YW?}$.5H(N*+vj6")

        print("Setting up your menu...")
        time.sleep(2)
        clear_output()

        ##MENU~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        while True:
            print("MENU")
            string = "1. Record information on appointment"
            for char in string:
                print(char, end='')
                time.sleep(.03)
            string = "\n2. Schedule appointment"
            for char in string:
                print(char, end='')
                time.sleep(.03)
            string = "\n3. Generate Report"
            for char in string:
                print(char, end='')
                time.sleep(.03) 
            string = "\n4. Quit\n"
            for char in string:
                print(char, end='')
                time.sleep(.03)

            ##user input
            choice = input()

            ##Record information of appointment------------------------------------------------------------------------
            if choice == "1":
                ##clearing the menu
                time.sleep(1)
                clear_output()

                ##creating the inputs menu
                print("Input Informarion of appointment:")
                
                ##Gathering Patients username
                PATIENTUNAME = input("Patient Username(Case Sensative): ")
    
                ##Gathering First Name
                PATIENTFNAME = input("First name of Patient: ")                
                PATIENTFNAME = PATIENTFNAME.title()
                
                ##Gathering Last Name
                PATIENTLNAME = input("Last name of Patient: ")
                PATIENTLNAME = PATIENTLNAME.title()
                
                ##Gathering Email
                PATIENTEMAIL = input("Email of Patient: ")
                PATIENTEMAIL = PATIENTEMAIL.title()
                
                ##Checking if Patient attended appointment
                while True:
                    PATIENTATTEND = input("Did the Patient attend?(y/n)")
                    if PATIENTATTEND.lower() == "y":
                        PATIENTATTEND = "Yes"
                        break
                    elif PATIENTATTEND.lower() =="n":
                        PATIENTATTEND = "No"
                        break
                    else:
                        print("Error: Invalid Input")
                        continue
                        
                ##Check if Patient was perscribed any medication
                PATIENTMEDS = input("Perscribed Medication(If none press enter)")
                
                ##Check Appointments Date
                APPOINTMENTDATE = input("Date of Appointment(mm/dd/yyyy)") 
                
                ##Check Appointments Time
                APPOINTMENTTIME = input("Time of Appointment (HH:MM in 24-hour clock)")
                
                ##connection to DB
                cursor = connection.cursor()
    
                ##try query to DB
                try:
                    query = f"SELECT * FROM APPOINTMENTS WHERE PATIENTUNAME = '{PATIENTUNAME}' AND PATIENTFNAME = '{PATIENTFNAME}' AND PATIENTLNAME = '{PATIENTLNAME}' AND PATIENTEMAIL = '{PATIENTEMAIL}' AND PATIENTATTEND = '{PATIENTATTEND}' \
                        AND PATIENTMEDS = '{PATIENTMEDS}' AND APPOINTMENTDATE = '{APPOINTMENTDATE}' AND APPOINTMENTTIME ='{APPOINTMENTTIME}' "
                    response = cursor.execute(query).fetchall()
                    if response:
                        continue # SKIP IF ALREADY EXISTS IN DATABASE
                    query = f"INSERT INTO APPOINTMENTS VALUES ('{PATIENTUNAME}','{PATIENTFNAME}','{PATIENTLNAME}','{PATIENTEMAIL}', '{PATIENTATTEND}', '{PATIENTMEDS}', '{APPOINTMENTDATE}','{APPOINTMENTTIME}')"
                    cursor.execute(query)
                    
                ##exception = query invalid
                except Exception as e:
                    logging.warning("Error updating Database:", e)
                    logging.warning(PATIENTUNAME, PATIENTFNAME, PATIENTLNAME, PATIENTEMAIL, PATIENTATTEND, PATIENTMEDS, APPOINTMENTDATE, APPOINTMENTTIME)

                
                try:
                    if PATIENTATTEND == 'Yes':
                        query = f"SELECT * FROM PATIENTS WHERE USERNAME = '{PATIENTUNAME}'"
                        response = cursor.execute(query).fetchall()
                        ##If it already exists in table, update patients w/ new table.
                        if response:
                            query = "UPDATE PATIENTS p \
             SET \
                LASTATTEND = ( \
                    SELECT \
                        TO_CHAR(MAX(TO_DATE(a.APPOINTMENTDATE, 'MM/DD/YYYY')), 'MM/DD/YYYY') \
                    FROM \
                        Appointments a \
                    WHERE \
                        a.PATIENTUNAME = p.USERNAME \
                )"
                            cursor.execute(query)
                    else:
                        query = "INSERT INTO Patients \
                        (USERNAME, NAME, EMAIL, MEDS, LASTATTEND) \
                        SELECT \
                            PATIENTUNAME, \
                            PATIENTFNAME || ' ' || PATIENTLNAME, \
                            PATIENTEMAIL, \
                            PATIENTMEDS, \
                            TO_CHAR(MAX(TO_DATE(APPOINTMENTDATE, 'MM/DD/YYYY')), 'MM/DD/YYYY') \
                        FROM \
                            Appointments \
                        WHERE NOT EXISTS ( \
                            SELECT 1 FROM Patients WHERE Patients.USERNAME = Appointments.PATIENTUNAME) \
                        GROUP BY \
                            PATIENTUNAME, PATIENTFNAME, PATIENTLNAME, PATIENTEMAIL, PATIENTMEDS"
                        cursor.execute(query)
                ##exception = query invalid
                except Exception as e:
                    logging.warning("Error updating Database:", e)
                ##commit the connection to DB    
                
                connection.commit()
                
                ##output and return to menu
                print("Returning to Menu")
                time.sleep(2)
                clear_output()

            ##Schedule Appointment-------------------------------------------------------------------------------------
            elif choice == "2":
                ##clearing the menu
                time.sleep(1)
                clear_output()

                ##creating the inputs menu
                print("Input Informarion of appointment:")
                
                ##Gathering Patients username
                PATIENTUNAME = input("Patient Username(Case Sensative): ")
                
                ##Gathering First Name
                PATIENTFNAME = input("First name of Patient: ")                
                PATIENTFNAME = PATIENTFNAME.title()
                
                ##Gathering Last Name
                PATIENTLNAME = input("Last name of Patient: ")
                PATIENTLNAME = PATIENTLNAME.title()
                
                ##Gathering Email
                PATIENTEMAIL = input("Email of Patient: ")
                PATIENTEMAIL = PATIENTEMAIL.title()
                
                ##Checking if Patient attended appointment
                while True:
                    PATIENTATTEND = input("Did the Patient attend?(y/n)")
                    if PATIENTATTEND.lower() == "y":
                        PATIENTATTEND = "Yes"
                        break
                    elif PATIENTATTEND.lower() =="n":
                        PATIENTATTEND = "No"
                        break
                    else:
                        print("Error: Invalid Input")
                        continue
                        
                ##Check if Patient was perscribed any medication
                PATIENTMEDS = input("Perscribed Medication(If none press enter)")
                
                ##Check Appointments Date
                APPOINTMENTDATE = input("Date of Appointment(mm/dd/yyyy)") 
                
                ##Check Appointments Time
                APPOINTMENTTIME = input("Time of Appointment (HH:MM in 24-hour clock)")
                
                
                cursor = connection.cursor()

                try:
                    query = f"SELECT * FROM APPOINTMENTS WHERE PATIENTUNAME = '{PATIENTUNAME}' AND PATIENTFNAME = '{PATIENTFNAME}' AND PATIENTLNAME = '{PATIENTLNAME}' AND PATIENTEMAIL = '{PATIENTEMAIL}' AND PATIENTATTEND = '{PATIENTATTEND}' \
                        AND PATIENTMEDS = '{PATIENTMEDS}' AND APPOINTMENTDATE = '{APPOINTMENTDATE}' AND APPOINTMENTTIME ='{APPOINTMENTTIME}' "
                    response = cursor.execute(query).fetchall()
                    if response:
                        continue # SKIP IF ALREADY EXISTS IN DATABASE
                    query = f"INSERT INTO APPOINTMENTS VALUES ('{PATIENTUNAME}','{PATIENTFNAME}','{PATIENTLNAME}','{PATIENTEMAIL}', '{PATIENTATTEND}', '{PATIENTMEDS}', '{APPOINTMENTDATE}','{APPOINTMENTTIME}')"
                    cursor.execute(query)
                except Exception as e:
                    logging.warning("Error updating Database:", e)
                    logging.warning(PATIENTUNAME, PATIENTFNAME, PATIENTLNAME, PATIENTEMAIL, PATIENTATTEND, PATIENTMEDS, APPOINTMENTDATE, APPOINTMENTTIME)
                
                try:
                    if PATIENTATTEND == 'Yes':
                        query = f"SELECT * FROM PATIENTS WHERE USERNAME = '{PATIENTUNAME}'"
                        response = cursor.execute(query).fetchall()
                        ##If it already exists in table, update patients w/ new table.
                        if response:
                            query = "UPDATE PATIENTS p \
             SET \
                LASTATTEND = ( \
                    SELECT \
                        TO_CHAR(MAX(TO_DATE(a.APPOINTMENTDATE, 'MM/DD/YYYY')), 'MM/DD/YYYY') \
                    FROM \
                        Appointments a \
                    WHERE \
                        a.PATIENTUNAME = p.USERNAME \
                )"
                            cursor.execute(query)
                    else:
                        query = "INSERT INTO Patients \
                        (USERNAME, NAME, EMAIL, MEDS, LASTATTEND) \
                        SELECT \
                            PATIENTUNAME, \
                            PATIENTFNAME || ' ' || PATIENTLNAME, \
                            PATIENTEMAIL, \
                            PATIENTMEDS, \
                            TO_CHAR(MAX(TO_DATE(APPOINTMENTDATE, 'MM/DD/YYYY')), 'MM/DD/YYYY') \
                        FROM \
                            Appointments \
                        WHERE NOT EXISTS ( \
                            SELECT 1 FROM Patients WHERE Patients.USERNAME = Appointments.PATIENTUNAME) \
                        GROUP BY \
                            PATIENTUNAME, PATIENTFNAME, PATIENTLNAME, PATIENTEMAIL, PATIENTMEDS"
                        cursor.execute(query)
                ##exception = query invalid
                except Exception as e:
                    logging.warning("Error updating Database:", e)
                ##commit the connection to DB    
                
                connection.commit()
                
                print("Returning to Menu")
                time.sleep(2)
                clear_output()

            ##Generate a report----------------------------------------------------------------------------------------
            elif choice == "3":
                ##clearing the menu
                time.sleep(1)
                clear_output()
                
                print("Report Generating...")
                
                rs = cursor.execute("SELECT * FROM APPOINTMENTS")
                df = pd.DataFrame(rs.fetchall())
                df.columns = ['Patients Username','Patients First Name' , 'Patients Last Name' , 'Patients Email' , 'Attendance' , 'Perscriptions' , 'Appointment Date', 'Appointment Time']
                
                filename = f'adminreport_{datetime.datetime.now().strftime("%H_%M_%m_%d_%Y")}.csv'
                df.to_csv(filename, index= True)
                
                time.sleep(1)
                print("Report Generated. See exported CSV.")
                time.sleep(1)
                print("Returning to Menu")
                time.sleep(2)
                clear_output()

            ##Quit-----------------------------------------------------------------------------------------------------
            elif choice == "4":
                ##clearing the menu
                time.sleep(1)
                clear_output()
                print("Logging you out...")
                time.sleep(2)
                print("Logged out. Have a great day")
                break

            ##Choice Not given-----------------------------------------------------------------------------------------
            else:
                print("ERROR: Invalid selection. Please select an option")
                time.sleep(1)
                clear_output() 

##RECORDS MANAGER CASE ✅==============================================================================================
    elif user.lower() == "records_manager":
        
        os.environ.get('TNS_ADMIN')
        connection = oracledb.connect(
            user='admin',
            password='8&n:f6=$uMrFp+j>',
            dsn = "ms73dyswf309gljx_high",
            config_dir="./config",
            wallet_location="./config",
            wallet_password="6YW?}$.5H(N*+vj6")        
        
        print("Setting up your menu...")
        time.sleep(2)
        clear_output()
        
        ##MENU~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        while True:
            print("MENU")
            string = "1. Create new users"
            for char in string:
                print(char, end='')
                time.sleep(.03)
            string = "\n2. Download Patient Information"
            for char in string:
                print(char, end='')
                time.sleep(.03)
            string = "\n3. Quit\n"
            for char in string:
                print(char, end='')
                time.sleep(.03)

            ##user input
            choice = input()
            
            ##adding new user to db------------------------------------------------------------------------------------
            if choice == "1":
                
                username = input("Patient's username: ")
                password = getpass.getpass("Patients Password:")

                cursor = connection.cursor()

                try:
                    query = f'CREATE USER {username} IDENTIFIED BY "{password}"'
                    cursor.execute(query)
                except Exception as e:
                    logging.warning("Error adding user to Database:", e)
                    break

                    
                print("User has been added to the Database")
                print("Please provide more information about the patient")
               
                ##gather name and capitalize
                name = input("Patient Name")
                name = name.title()
                
                ##gather patient address
                address = input("Patients Address")
                
                ##gather patient email
                email = input("Patients Email Address")
                
                ##gather the patients medications if applicable
                meds = input("Patients current medications(press enter if none apply)")
                
                ##assuming that the first appointment is the date that user is being input
                ##so I will be hard coding in the date that the user is input in the system
                lastattend = date.today().strftime('%m/%d/%Y') 
                
                cursor = connection.cursor()
                
                ##attempt to insert values into the db
                try:
                    query = f"INSERT INTO PATIENTS VALUES ('{username}','{name}','{address}','{email}','{meds}','{lastattend}')"
                    cursor.execute(query)
                except Exception as e:
                    logging.warning("Error inserting Patient:", e)
                
                print("Returning to Menu")
                time.sleep(2)
                clear_output()
                
            ##Download Patient Info------------------------------------------------------------------------------------
            elif choice == "2":
                
                username = input("Patient's username: ")
                time.sleep(1)
                print("Downloading Patient Information. One moment Please")
                time.sleep(3)
                
                cursor = connection.cursor()

                try:
                    query = f"SELECT * FROM PATIENTS WHERE USERNAME = '{username}'"
                    rs = cursor.execute(query)
                    df = pd.DataFrame(rs.fetchall())
                    df.columns = ['Patient Username','Patients Name' , 'Patients ADDRESS' , 'Patients Email' , 'Patients Medications' , 'Last Attendance Date']
                except Exception as e:
                    logging.warning("Error updating Database:", e)

                connection.commit()
                
                filename = f'patientlookup_{username}_{datetime.datetime.now().strftime("%H_%M_%m_%d_%Y")}.csv'
                df.to_csv(filename, index= True)
                time.sleep(1)
                print("CSV exported.")
                time.sleep(1)
                print("Returning to Menu")
                time.sleep(2)
                clear_output()
            ##Quit-----------------------------------------------------------------------------------------------------
            elif choice == "3":
                ##clearing the menu
                time.sleep(1)
                clear_output()
                print("Logging you out...")
                time.sleep(2)
                print("Logged out. Have a great day")
                break
        
            ##choice not given-----------------------------------------------------------------------------------------
            else:
                print("ERROR: Invalid selection. Please select an option")
                time.sleep(1)
                clear_output()

##CLINICAL STAFF CASE ✅===============================================================================================
    elif user.lower() == "clinicalstaff":
        
        os.environ.get('TNS_ADMIN')
        connection = oracledb.connect(
            user='admin',
            password='8&n:f6=$uMrFp+j>',
            dsn = "ms73dyswf309gljx_high",
            config_dir="./config",
            wallet_location="./config",
            wallet_password="6YW?}$.5H(N*+vj6")
        
        print("Setting up your menu...")
        time.sleep(2)
        clear_output()
        
        ##MENU~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        while True:
            print("MENU")
            string = "1. Look up Patient "
            for char in string:
                print(char, end='')
                time.sleep(.03)
            string = "\n2. Modify Patient Information"
            for char in string:
                print(char, end='')
                time.sleep(.03)
            string = "\n3. Quit\n"
            for char in string:
                print(char, end='')
                time.sleep(.03)

            ##Selecting what user wants to do  
            choice = input()
            
            ##Lookup Patient Info--------------------------------------------------------------------------------------
            if choice == "1":
                ##clearing the menu
                time.sleep(1)
                clear_output()
                
                ##Gather Username
                USERNAME = input("Patients Username(Case Sensative): ")
                
                print("Searching the Database, one moment please")
                time.sleep(3)
                clear_output()

                cursor = connection.cursor()

                try:
                    query = f"SELECT * FROM PATIENTS WHERE USERNAME = '{USERNAME}'"
                    rs = cursor.execute(query)
                    df = pd.DataFrame(rs.fetchall())
                    df.columns = ['Patient Username','Patients Name' , 'Patients ADDRESS' , 'Patients Email' , 'Patients Medications' , 'Last Attendance Date']
                    display(df)
                except Exception as e:
                    logging.warning("Error updating Database:", e)

                connection.commit()
                
                string = "\n1. Return to Menu "
                for char in string:
                    print(char, end='')
                    time.sleep(.03)
                string = "\n2. Export as csv"
                for char in string:
                    print(char, end='')
                    time.sleep(.03)
                string = "\n3. Exit and Logout\n"
                for char in string:
                    print(char, end='')
                    time.sleep(.03)
                    
                ##Selecting what user wants to do  
                choice = input()
                
                ##return to the menu-----------------------------------------------------------------------------------
                if choice == '1':
                    print("Returning to Menu")
                    time.sleep(2)
                    clear_output()
                    
                ##exporting the data-----------------------------------------------------------------------------------
                elif choice == '2':
                    filename = f'patientlookup_{datetime.datetime.now().strftime("%H_%M_%m_%d_%Y")}.csv'
                    df.to_csv(filename, index= True)
                    time.sleep(1)
                    print("CSV exported.")
                    time.sleep(1)
                    print("Returning to Menu")
                    time.sleep(2)
                    clear_output()
                    
                ##quit-------------------------------------------------------------------------------------------------
                elif choice == '3':
                    ##clearing the menu
                    time.sleep(1)
                    clear_output()
                    print("Logging you out...")
                    time.sleep(2)
                    print("Logged out. Have a great day")
                    break   
                    
                ##choice not given-------------------------------------------------------------------------------------
                else:
                    print("ERROR: Invalid selection. Please select an option")
                    time.sleep(1)
                    clear_output()

                    
            ##Modify Patient Information-------------------------------------------------------------------------------
            elif choice == "2":
                ##clearing the menu
                time.sleep(1)
                clear_output()
                
                USERNAME = input("Please provide the Username(case sensitive) of the patient you wish to view")
                print("One moment while the database is queried")

                try:
                    query = f"SELECT * FROM PATIENTS WHERE USERNAME = '{USERNAME}'"
                    rs = cursor.execute(query)
                    df = pd.DataFrame(rs.fetchall())
                    df.columns = ['Patient Username','Patients Name' , 'Patients ADDRESS' , 'Patients Email' , 'Patients Medications' , 'Last Attendance Date']
                    display(df)
                except Exception as e:
                    logging.warning("Error locating in Database:", e)

                connection.commit()
                
                ##can only update patients by adding appointments, so I'll give option 
                ##to just return to menu incase of accidental selection
                string = "\n1. Update Email Address"
                for char in string:
                    print(char, end='')
                    time.sleep(.03)
                string = "\n2. Update Address"
                for char in string:
                    print(char, end='')
                    time.sleep(.03)
                string = "\n3. Update Medications"
                for char in string:
                    print(char, end='')
                    time.sleep(.03)
                string = "\n4. Return to Menu"
                for char in string:
                    print(char, end='')
                    time.sleep(.03)
                    
                ##Selecting what user wants to do  
                choice = input()
                
                ##Update Email-----------------------------------------------------------------------------------------
                if choice == '1':
                    email = input("Enter the new Email Address: ")           
                    email = email.title()
                    
                    lastattend = input("Date of Appointment(mm/dd/yyyy)")
                    
                    try:
                        query = f"UPDATE PATIENTS SET EMAIL = '{email}', LASTATTEND = '{lastattend}' WHERE USERNAME = '{USERNAME}'"
                        cursor.execute(query)
                    except Exception as e:
                        logging.warning("Error Updating User: ", e)
                    
                    connection.commit()

                    time.sleep(2)
                    clear_output()
                    
                ##Update Address---------------------------------------------------------------------------------------
                elif choice == '2':
                    address = input("Enter the new Address: ")           
                    
                    lastattend = input("Date of Appointment(mm/dd/yyyy)")
                    
                    try:
                        query = f"UPDATE PATIENTS SET ADDRESS = '{address}', LASTATTEND = '{lastattend}' WHERE USERNAME = '{USERNAME}'"
                        cursor.execute(query)
                    except Exception as e:
                        logging.warning("Error Updating User: ", e)
                    
                    connection.commit()
                    
                    time.sleep(2)
                    clear_output()
                
                ##update medications-----------------------------------------------------------------------------------
                elif choice == '3':
                    meds = input("Perscribed Medication(If none press enter)")           
                    
                    lastattend = input("Date of Appointment(mm/dd/yyyy)")
                    
                    try:
                        query = f"UPDATE PATIENTS SET MEDS = '{meds}', LASTATTEND = '{lastattend}' WHERE USERNAME = '{USERNAME}'"
                        cursor.execute(query)
                    except Exception as e:
                        logging.warning("Error Updating User: ", e)

                    connection.commit()

                    time.sleep(2)
                    clear_output()
                
                ##return to menu---------------------------------------------------------------------------------------
                elif choice == '4':
                    print("Returning to Menu")
                    time.sleep(2)
                    clear_output()
                    
                ##choice not given-------------------------------------------------------------------------------------
                else:
                    print("ERROR: Invalid selection. Please select an option")
                    time.sleep(1)
                    clear_output()
                
                print("Returning to Menu")
                time.sleep(2)
                clear_output()   
                
            ##Quit-----------------------------------------------------------------------------------------------------
            elif choice == "3":
                ##clearing the menu
                time.sleep(1)
                clear_output()
                print("Logging you out...")
                time.sleep(2)
                print("Logged out. Have a great day")
                break
        
            ##choice not given-----------------------------------------------------------------------------------------
            else:
                print("ERROR: Invalid selection. Please select an option")
                time.sleep(1)
                clear_output()


##USERS CASE ✅========================================================================================================       
    else:
        os.environ.get('TNS_ADMIN')
        connection = oracledb.connect(
            user='admin',
            password='8&n:f6=$uMrFp+j>',
            dsn = "ms73dyswf309gljx_high",
            config_dir="./config",
            wallet_location="./config",
            wallet_password="6YW?}$.5H(N*+vj6")  
        
        ##MENU FOR RECORDS MANAGERS
        print("Setting up your menu...")
        time.sleep(2)
        clear_output()
        
        ##printing out Menu
        while True:
            print("MENU")
            string = "1. View Appointment Records"
            for char in string:
                print(char, end='')
                time.sleep(.03)
            string = "\n2. Download Appointment Records"
            for char in string:
                print(char, end='')
                time.sleep(.03)
            string = "\n3. View Your Information"
            for char in string:
                print(char, end='')
                time.sleep(.03) 
            string = "\n4. Quit\n"
            for char in string:
                print(char, end='')
                time.sleep(.03)

            ##Selecting what user wants to do  
            choice = input()
            
            ##viewing data from Appointments---------------------------------------------------------------------------
            if choice == "1":
                cursor = connection.cursor()
                rs = cursor.execute(f"SELECT * FROM APPOINTMENTS WHERE PATIENTUNAME = '{user}'")
                df = pd.DataFrame(rs.fetchall())
                df.columns = ['Patients Username','Patients First Name' , 'Patients Last Name' , 'Patients Email' , 'Attendance' , 'Perscriptions' , 'Appointment Date', 'Appointment Time']
                display(df)
                choice = input("Press Enter to go back to Menu")
                print("Returning to Menu")
                time.sleep(2)
                clear_output() 
                
            ##exporting data from appointments-------------------------------------------------------------------------
            elif choice == "2":
                cursor = connection.cursor()
                rs = cursor.execute(f"SELECT * FROM APPOINTMENTS WHERE PATIENTUNAME = '{user}'")
                df = pd.DataFrame(rs.fetchall())
                df.columns = ['Patients Username','Patients First Name' , 'Patients Last Name' , 'Patients Email' , 'Attendance' , 'Perscriptions' , 'Appointment Date', 'Appointment Time']

                filename = f'{user}_data_{datetime.datetime.now().strftime("%H_%M_%m_%d_%Y")}.csv'
                df.to_csv(filename, index= True)
                print("See downloads for file")
                time.sleep(2)
                print("Returning to Menu")
                time.sleep(2)
                clear_output()                
            ##view patient information---------------------------------------------------------------------------------
            elif choice == "3":
                cursor = connection.cursor()
                rs = cursor.execute(f"SELECT * FROM PATIENTS WHERE USERNAME = '{user}'")
                df = pd.DataFrame(rs.fetchall())
                df.columns = ['Username','Name' , 'Email' , 'Address' , 'Medications' , 'Most Recent Appointment']
                display(df)
                choice = input("Press Enter to go back to Menu")
                print("Returning to Menu")
                time.sleep(2)
                clear_output()
            ##Quit-----------------------------------------------------------------------------------------------------
            elif choice == "4":
                ##clearing the menu
                time.sleep(1)
                clear_output()
                print("Logging you out...")
                time.sleep(2)
                print("Logged out. Have a great day")
                break
            ##choice not given-----------------------------------------------------------------------------------------
            else:
                print("ERROR: Invalid selection. Please select an option")
                time.sleep(1)
                clear_output()

You have reached the maximum number of attempts allowed.
Please enter your email address so that you can reset your password: logan.lauton@marquette.edu
Check you email
